# 4.針對顧客所屬地區的經緯度，將顧客依地理位置分群，並比較不同群組特徵(例如 : 性別、年齡、 婚姻、扶養人數等)的差異性，針對其中任一群組，再建立其群組使用公司服務的關聯規則。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.cluster import KMeans ,DBSCAN
from sklearn import cluster,metrics
from mlxtend.frequent_patterns import apriori ,association_rules

sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

# EDA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')

df

# Processing Data

In [ ]:
location_df = df[['緯度' ,'經度']]
x = location_df

# 評估指標

# 輪廓係數(Silhouette Evaluation)

輪廓係數的值越接近1，表示聚類的效果越好

In [ ]:
silhouette_avg = []
for i in range(2,9):
    kmeans_fit = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=10,random_state=0).fit(x)
    silhouette_avg.append(metrics.silhouette_score(x, kmeans_fit.labels_))
print(f"輪廓係數 = {silhouette_avg}")

plt.plot(range(2, 9), silhouette_avg, 'bx-')
plt.title('silhouette')
plt.xlabel('No of clusters')
plt.ylabel('Avg')
plt.show()

圖表得知聚類數量為2時，輪廓係數最高，然後在聚類數量為3時仍然保持相對高的水平這表示將數據分為2或3個聚類可能是合適的

KMeans可以使用 n_clusters=2 or 3 作為 KMeans 模型的參數

# WCSS 損失函數 (within-cluster sum of squares)

Elbow method，手肘法
透過嘗試多種類別個數，並將相應的 WCSS 記錄下來並且畫出，理想上最佳的K值應該是最大轉折處

In [ ]:
wcss = []
for i in range(1, 9):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(x)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 9), wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('No of clusters')
plt.ylabel('WCSS')
# 在每個點上標示相應的群數
for k, w in zip(range(1, 9), wcss):
    plt.text(k, w, f'({k},{int(w)})', ha='center', va='bottom')
plt.show()

聚類數量從1到8變化時，WCSS逐漸減小。肘部通常是曲線開始彎曲的地方我們大概是在聚類數量為3或4時，出現肘部，WCSS開始減少的速度變慢


結合上面的輪廓係數評估，我們認為聚類數量為3是一個合適的選擇

# KMeans Clustering

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
df['Cluster'] = kmeans.fit_predict(location_df)

plt.figure(figsize=(12, 6))
sns.scatterplot(x='經度', y='緯度', hue='Cluster', data=df, palette='viridis', legend='full')
plt.title('Customer Clusters Based on Geographic Location')
plt.show()

圖表顯示了美國西岸的加利福尼亞州大部分地區，包括洛杉磯、舊金山、聖地牙哥、薩克拉門托等主要城市

可得知加利福尼亞州的客戶群落主要分佈在沿海地區，尤其是在洛杉磯、舊金山和聖地牙哥等大城市周圍

In [ ]:
# 性別
sns.countplot(x='Cluster', hue='性別', data=df, palette='viridis')
plt.title('Gender Distribution in Clusters')
plt.show()

In [ ]:
# 年齡
sns.boxplot(x='Cluster', y='年齡', data=df)
plt.title('Age Distribution in Clusters')
plt.show()

In [ ]:
# 婚姻
sns.countplot(x='Cluster', hue='婚姻', data=df, palette='viridis')
plt.title('Marital Status Distribution in Clusters')
plt.show()

In [ ]:
# 扶養人數
sns.boxplot(x='Cluster', y='扶養人數', data=df)
plt.title('Number of Dependents in Clusters')
plt.show()

# Selection Cluster

In [ ]:
# 挑選分群結果為 0 的資料
selected_cluster = df[df['Cluster'] == 0]

# 選取公司服務的欄位
selected_service = selected_cluster[['網路連線類型', '優惠方式']]

# Association rule

In [ ]:
# One-hot Encoding
one_hot_eccoder = pd.get_dummies(selected_service)
print(one_hot_eccoder.head())

In [ ]:
# Apriori Algorithm
frequent_itemsets = apriori(one_hot_eccoder, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

print(rules)
rules.to_csv('FR4_rules.csv', index=False, encoding='utf_8_sig')